## Введение в обработку естественного языка

### Урок 7. Сверточные нейронные сети для анализа текста

Задание
Берем отызывы за лето (из архива с материалами или предыдущего занятия)
1. Учим conv сеть для классификации
2. Рассмотреть 2-а варианта сеточек
2.1 Инициализировать tf.keras.layers.Embedding предобученными векторами взять к примеру с https://rusvectores.org/ru/
2.2 Инициализировать слой tf.keras.layers.Embedding по умолчанию (ну то есть вам ничего не делать с весами)

Сравнить две архитектуры с предобученными весами и когда tf.keras.layers.Embedding обучается сразу со всей сеточкой, что получилось лучше

In [80]:
pip install --upgrade xlrd

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [81]:
pip install pymorphy2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [82]:
pip install stop-words

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [83]:
import numpy as np

import pandas as pd

from string import punctuation

from stop_words import get_stop_words

from pymorphy2 import MorphAnalyzer

import re

from sklearn.model_selection import train_test_split

import nltk
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist
nltk.download("punkt")

import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Input, Embedding, Conv1D, GlobalMaxPool1D
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.callbacks import TensorBoard 
from keras.callbacks import EarlyStopping  
from keras.utils import np_utils

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


**Произведем загрузку данных.**

In [84]:
df = pd.read_excel('./Отзывы за лето.xls')

df.head()

,Rating,Content,Date
0,5,It just works!,2017-08-14
1,4,В целом удобноное приложение...из минусов хотя...,2017-08-14
2,5,Отлично все,2017-08-14
3,5,Стал зависать на 1% работы антивируса. Дальше ...,2017-08-14
4,5,"Очень удобно, работает быстро.",2017-08-14


**Проведем предобработку текста.**

In [85]:
sw = set(get_stop_words("ru"))

exclude = set(punctuation)

morpher = MorphAnalyzer()

def preprocess_text(txt):
    txt = str(txt)
    txt = "".join(c for c in txt if c not in exclude)
    txt = txt.lower()
    txt = re.sub("не\s", "не", txt)
    txt = [morpher.parse(word)[0].normal_form for word in txt.split() if word not in sw]
    return " ".join(txt)

df['Content'] = df['Content'].apply(preprocess_text)

**Разделим выборку на тренировочную, тестовую и валидационную.**

In [86]:
train_df, test_val_df = train_test_split(df, test_size=0.33, random_state=42)

test_df, val_df = train_test_split(test_val_df, test_size=0.5, random_state=42)

In [87]:
train_corpus = " ".join(train_df['Content'])

train_corpus = train_corpus.lower()

In [88]:
tokens = word_tokenize(train_corpus)

**Насторим параметры сети.**

In [89]:
max_words = 1000

max_len = 100

num_classes = 6

**Настроим параметры обучения.**

In [90]:
epochs = 20

batch_size = 512

print_batch_n = 100

**Отсортируем и отберем самые частотные токены.**

In [91]:
tokens_filtered = [word for word in tokens if word.isalnum()]

dist = FreqDist(tokens_filtered)

tokens_filtered_top = [pair[0] for pair in dist.most_common(max_words-1)]

In [92]:
vocabulary = {v: k for k, v in dict(enumerate(tokens_filtered_top, 1)).items()}

In [93]:
def text_to_sequence(text, maxlen):
    result = []
    tokens = word_tokenize(text.lower())
    tokens_filtered = [word for word in tokens if word.isalnum()]
    for word in tokens_filtered:
        if word in vocabulary:
            result.append(vocabulary[word])
    padding = [0]*(maxlen-len(result))
    return padding + result[-maxlen:]

In [94]:
x_train = np.asarray([text_to_sequence(text, max_len) for text in train_df['Content']], dtype=np.int32)

x_test = np.asarray([text_to_sequence(text, max_len) for text in test_df['Content']], dtype=np.int32)

x_val = np.asarray([text_to_sequence(text, max_len) for text in val_df['Content']], dtype=np.int32)

In [95]:
y_train = keras.utils.np_utils.to_categorical(train_df['Rating'], num_classes)

y_test = keras.utils.np_utils.to_categorical(test_df['Rating'], num_classes)

y_val = keras.utils.np_utils.to_categorical(val_df['Rating'], num_classes)

**Создадим сеть.**

In [96]:

model = Sequential()
model.add(Embedding(input_dim=max_words, output_dim=128, input_length=max_len))
model.add(Conv1D(128, 3))
model.add(Activation("relu"))
model.add(GlobalMaxPool1D())
model.add(Dense(10))
model.add(Activation("relu"))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

In [97]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [98]:
tensorboard=TensorBoard(log_dir='./logs', write_graph=True, write_images=True)

early_stopping=EarlyStopping(monitor='val_loss')  

history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[tensorboard, early_stopping])

Epoch 1/20
25/25 [==============================] - 17s 652ms/step - loss: 1.3655 - accuracy: 0.6842 - val_loss: 1.0711 - val_accuracy: 0.7011
Epoch 2/20
25/25 [==============================] - 19s 744ms/step - loss: 0.9124 - accuracy: 0.7188 - val_loss: 0.8398 - val_accuracy: 0.7307


In [99]:
score = model.evaluate(x_val, y_val, batch_size=batch_size, verbose=1)

print('\n')

print('Test score:', score[0])

print('Test accuracy:', score[1])

7/7 [==============================] - 1s 95ms/step - loss: 0.7973 - accuracy: 0.7471


Test score: 0.7973234057426453
Test accuracy: 0.7471399307250977


In [100]:
results = model.predict(x_test, batch_size=batch_size, verbose=1)

7/7 [==============================] - 1s 103ms/step


**Инициализируем Embedding слой с предобученными векторами.**

In [101]:
!wget https://rusvectores.org/static/models/rusvectores4/taiga/taiga_upos_skipgram_300_2_2018.vec.gz

!gunzip taiga_upos_skipgram_300_2_2018.vec.gz

--2022-09-21 18:16:18--  https://rusvectores.org/static/models/rusvectores4/taiga/taiga_upos_skipgram_300_2_2018.vec.gz
Resolving rusvectores.org (rusvectores.org)... 116.203.104.23
Connecting to rusvectores.org (rusvectores.org)|116.203.104.23|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 346385366 (330M) [application/x-gzip]
Saving to: ‘taiga_upos_skipgram_300_2_2018.vec.gz.1’

taiga_upos_skipgram 100%[===================>] 330.34M  11.8MB/s    in 30s     

2022-09-21 18:16:50 (11.0 MB/s) - ‘taiga_upos_skipgram_300_2_2018.vec.gz.1’ saved [346385366/346385366]

gzip: taiga_upos_skipgram_300_2_2018.vec already exists; do you wish to overwrite (y or n)? n
	not overwritten


In [102]:
tokenizer = Tokenizer()

tokenizer.fit_on_texts(train_df['Content'])

vocab_size = len(tokenizer.word_index) + 1

In [103]:
embeddings_index = {}

with open('taiga_upos_skipgram_300_2_2018.vec') as f:
    for line in f:
        word, coefs = line.split(maxsplit=1)
        coefs = np.fromstring(coefs, "f", sep=" ")
        word = word.split('_')[0]
        embeddings_index[word] = coefs

print("Found %s word vectors." % len(embeddings_index))

Found 209666 word vectors.


In [104]:
embedding_dim = 300

hits = 0

misses = 0

embedding_matrix = np.zeros((vocab_size, embedding_dim))
for word, i in tokenizer.word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector
        hits += 1
    else:
        misses += 1
        
print("Converted %d words (%d misses)" % (hits, misses))

Converted 4814 words (4703 misses)


In [105]:
model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=300, input_length=max_len, weights=[embedding_matrix],
    trainable=False))
model.add(Conv1D(128, 3))
model.add(Activation("relu"))
model.add(GlobalMaxPool1D())
model.add(Dense(10))
model.add(Activation("relu"))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

In [106]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [107]:
tensorboard=TensorBoard(log_dir='./logs', write_graph=True, write_images=True)

early_stopping=EarlyStopping(monitor='val_loss')  

history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[tensorboard, early_stopping])

Epoch 1/20
25/25 [==============================] - 15s 586ms/step - loss: 1.4156 - accuracy: 0.6699 - val_loss: 1.1291 - val_accuracy: 0.7011
Epoch 2/20
25/25 [==============================] - 15s 582ms/step - loss: 0.9714 - accuracy: 0.7226 - val_loss: 0.9075 - val_accuracy: 0.7394


In [108]:
score = model.evaluate(x_val, y_val, batch_size=batch_size, verbose=1)

print('\n')

print('Test score:', score[0])

print('Test accuracy:', score[1])

7/7 [==============================] - 2s 213ms/step - loss: 0.8547 - accuracy: 0.7568


Test score: 0.8546596169471741
Test accuracy: 0.7568202018737793


In [109]:
results = model.predict(x_test, batch_size=batch_size, verbose=1)

7/7 [==============================] - 2s 217ms/step


**Качество немного возросло.**